<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# ПРОЕКТ-2: Анализ вакансий из HeadHunter
   

In [1]:
# импорт pandas и psycopg
import pandas as pd
import psycopg2

In [3]:
# создаем соединение
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Количество вакансий в базе данных 

In [4]:
# считаем по количеству id вакансий в таблице vacancies
query_3_1 = f'''select count(v.id) as vacancies_count
                from vacancies as v
            '''

In [5]:
df_3_1 = pd.read_sql_query(query_3_1, connection)
df_3_1

C:\Users\levte\AppData\Local\Temp\ipykernel_27752\2127050470.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_3_1 = pd.read_sql_query(query_3_1, connection)


,vacancies_count
0,49197


2. Общее количество работодателей 

In [6]:
# считаем по количеству id работодателей в таблице employers
query_3_2 = f'''select count(e.id) as employers_count
                from employers as e
            '''

In [7]:
df_3_2 = pd.read_sql_query(query_3_2, connection)
df_3_2

C:\Users\levte\AppData\Local\Temp\ipykernel_27752\2741247974.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_3_2 = pd.read_sql_query(query_3_2, connection)


,employers_count
0,23501


3. Общее количество регионов

In [8]:
# считаем по количеству id районов в таблице areas
query_3_3 = f'''select count(a.id) as areas_count
                from areas as a
            '''

In [9]:
df_3_3 = pd.read_sql_query(query_3_3, connection)
df_3_3

C:\Users\levte\AppData\Local\Temp\ipykernel_27752\3006110907.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_3_3 = pd.read_sql_query(query_3_3, connection)


,areas_count
0,1362


4. Общее количество сфер деятельности

In [10]:
# считаем по количеству id сфер деятельности в таблице industries
query_3_4 = f'''select count(i.id) as industries_count
                from industries as i
            '''

In [11]:
df_3_4 = pd.read_sql_query(query_3_4, connection)
df_3_4

C:\Users\levte\AppData\Local\Temp\ipykernel_27752\764330752.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_3_4 = pd.read_sql_query(query_3_4, connection)


,industries_count
0,294


***

Выводы по предварительному анализу данных:
1. База данных является достаточно обширной.
2. Портал пользуется спросом не только в РФ, но и за рубежом.
3. У одного работодателя может быть несколько сфер деятельности.

# Юнит 4. Детальный анализ вакансий

1. ТОП-5 регионов по количеству вакансий

In [12]:
# cоединяем таблицу areas с таблицей vacancies через left join по id региона
# считаем по количеству id вакансий 
# группируем по названию районов
# сортируем по количеству вакансий в порядке убывания
# выводим первые пять строк
query_4_1 = f'''select
                    a.name as area,
                    count(v.id) as cnt
                from areas as a
                left join vacancies as v on v.area_id = a.id
                group by area
                order by cnt desc
                limit 5
            '''

In [13]:
df_4_1 = pd.read_sql_query(query_4_1, connection)
df_4_1

C:\Users\levte\AppData\Local\Temp\ipykernel_27752\1522353868.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_4_1 = pd.read_sql_query(query_4_1, connection)


,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


2. Количество вакансий, у которых заполнено хотя бы одно из двух полей с зарплатой

In [14]:
# создаем условие, при котором в рассчет берутся только те строки, 
# в которых заполнено хотя бы одно из полей с зарплатой в таблице vacancies,
# считаем по количеству id вакансий
query_4_2 = f'''select count(v.id) as vacancies_count
                from vacancies as v
                where 
                    (v.salary_from is not null and v.salary_to is null)
                    or (v.salary_from is null and v.salary_to is not null)
                    or (v.salary_from is not null and v.salary_to is not null)
            '''

In [15]:
df_4_2 = pd.read_sql_query(query_4_2, connection)
df_4_2

C:\Users\levte\AppData\Local\Temp\ipykernel_27752\3761913292.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_4_2 = pd.read_sql_query(query_4_2, connection)


,vacancies_count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [16]:
# вычисляем средние значения нижней и верхней границ зарплатной вилки из таблицы vacancies
# округляем результат до целого значения
query_4_3 = f'''select 
                    round(avg(v.salary_from)) as salary_from_avg,
                    round(avg(v.salary_to)) as salary_to_avg
                from vacancies as v
            '''

In [17]:
df_4_3 = pd.read_sql_query(query_4_3, connection)
df_4_3

C:\Users\levte\AppData\Local\Temp\ipykernel_27752\4169514055.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_4_3 = pd.read_sql_query(query_4_3, connection)


,salary_from_avg,salary_to_avg
0,71065.0,110537.0


4. Количество вакансий для каждого сочетания типа рабочего графика и типа трудоустройства

In [18]:
# считаем по количеству id вакансий в таблице vacancies 
# группируем сначала по типу рабочего графика, потом по типу трудоустройства
# сортируем по количеству вакансий в порядке убывания
query_4_4 = f'''select
                    v.schedule,
                    v.employment,
                    count(v.id) as cnt
                from vacancies as v
                group by v.schedule, v.employment
                order by cnt desc
            '''

In [19]:
df_4_4 = pd.read_sql_query(query_4_4, connection)
df_4_4

C:\Users\levte\AppData\Local\Temp\ipykernel_27752\1636312543.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_4_4 = pd.read_sql_query(query_4_4, connection)


,schedule,employment,cnt
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Значение поля "Требуемый опыт работы" (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта

In [20]:
# считаем по количеству id вакансий в таблице vacancies
# делаем группировку по опыту работы
# сортируем по количеству вакансий в порядке возрастания
query_4_5 = f'''select v.experience
                from vacancies as v
                group by v.experience
                order by count(id)
            '''

In [21]:
df_4_5 = pd.read_sql_query(query_4_5, connection)
df_4_5

C:\Users\levte\AppData\Local\Temp\ipykernel_27752\802873672.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_4_5 = pd.read_sql_query(query_4_5, connection)


,experience
0,Более 6 лет
1,Нет опыта
2,От 3 до 6 лет
3,От 1 года до 3 лет


***

Выводы по детальному анализу данных:
1. Предсказуемо, большее число вакансий в Москве - более 10%. Однако, стоить отметить, что в ТОП-5 по количеству вакансий, размещенных на сайте, также объявления из Беларуси (Минск) и Казахстана (Алматы).
2. Почти у половины вакансий нет данных о предлагаемой зарплате, что не дает возможности провести достаточный анализ по зарплатам.
3. Более 70% вакансий предполагают полный рабочий день и полную занятость. Однако, высока доля и удаленной работы по всем видам трудоустройства - почти 20%.
4. В половине случаев требуется опыт работы от 1 до 3 лет.

# Юнит 5. Анализ работодателей

1. Работодатели на первом и пятом местах по количеству вакансий

In [22]:
# Таблица 1:
# cоединяем таблицу vacancies с таблицей employers через left join по id работодателя
# считаем по количеству id вакансий 
# группируем по id работодателей
# сортируем по количеству вакансий в порядке убывания
# выводим только первую строку

# Таблица 2: производим тот же набор операций, только при этом отсекаем первые четыре строки и выводим только пятую

# объединяем две таблицы через union all 

query_5_1 = f'''(select 
                    e.name,
                    count(v.id) as cnt
                from vacancies as v
                left join employers as e on v.employer_id = e.id
                group by e.id
                order by cnt desc
                limit 1)
                
                union all
                
                (select 
                    e.name,
                    count(v.id)
                from vacancies as v
                left join employers as e on v.employer_id = e.id
                group by e.id
                order by count(v.id) desc
                limit 1
                offset 4)
            '''

In [23]:
df_5_1 = pd.read_sql_query(query_5_1, connection)
df_5_1

C:\Users\levte\AppData\Local\Temp\ipykernel_27752\556590240.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_5_1 = pd.read_sql_query(query_5_1, connection)


,name,cnt
0,Яндекс,1933
1,Газпром нефть,331


2. Регион с наибольшим количеством работодателей, в котором нет вакансий

In [24]:
# cоединяем таблицу areas с таблицей vacancies через left join по id региона
# затем полученную таблицу соединяем с таблицей emplyers через left join по id региона
# считаем по количеству id работодателей 
# считаем по количеству id вакансий с null значением 
# группируем по id региона
# сортируем по количеству работодателей в порядке убывания
# выводим только первую строку

query_5_2 = f'''select
                    a.name,
                    count(e.id) as cnt_employers,
                    count(v.id) as cnt_vacancies
                from areas as a
                left join vacancies as v on a.id = v.area_id
                left join employers as e on a.id = e.area
                where v.id is null
                group by a.id
                order by cnt_employers desc
                limit 1
            '''

In [25]:
df_5_2 = pd.read_sql_query(query_5_2, connection)
df_5_2

C:\Users\levte\AppData\Local\Temp\ipykernel_27752\762398461.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_5_2 = pd.read_sql_query(query_5_2, connection)


,name,cnt_employers,cnt_vacancies
0,Россия,410,0


3. Работодатель с наибольшим количеством регионов, в которых он публикует свои вакансии


In [26]:
# cоединяем таблицу employers с таблицей vacancies через left join по id работодателя
# считаем по количеству уникальных id регионов
# группируем по id работодателя
# сортируем по количеству уникальных регионов
# выводим только первую строку
query_5_3 = f'''select
                    e.name,
                    count(distinct v.area_id) as cnt_area
                from employers as e 
                left join vacancies as v on e.id = v.employer_id
                group by e.id
                order by cnt_area desc
                limit 1
            '''

In [27]:
df_5_3 = pd.read_sql_query(query_5_3, connection)
df_5_3

C:\Users\levte\AppData\Local\Temp\ipykernel_27752\3556584873.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_5_3 = pd.read_sql_query(query_5_3, connection)


,name,cnt_area
0,Яндекс,181


4. Количество работодателей, у которых не указана сфера деятельности 

In [28]:
# cоединяем таблицу employers с таблицей employers_industries через left join по id работодателя
# cоединяем полученную таблицу с таблицей industries через left join по id сферы деятельности
# считаем по количеству id работодателей, у которых не указана сфера деятельности
query_5_4 = f'''select count(e.id)
                from employers as e 
                left join employers_industries as ei on e.id = ei.employer_id
                left join industries as i on ei.industry_id = i.id
                where i.name is null
            '''

In [29]:
df_5_4 = pd.read_sql_query(query_5_4, connection)
df_5_4

C:\Users\levte\AppData\Local\Temp\ipykernel_27752\2345892774.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_5_4 = pd.read_sql_query(query_5_4, connection)


,count
0,8419


5. Компания, находящаяся на третьем месте в алфавитном списке (по названию) компаний, у которой указано четыре сферы деятельности

In [30]:
# cоединяем таблицу employers с таблицей employers_industries через left join по id работодателя
# cоединяем полученную таблицу с таблицей industries через left join по id сферы деятельности
# группируем по id работодателей, у которых количество уникальных сфер деятельности равно 4
# сортируем по имени работодателей в алфавитном порядке
# отсекаем первые две строки и выводим только третью
query_5_5 = f'''select e.name
                from employers as e 
                left join employers_industries as ei on e.id = ei.employer_id
                left join industries as i on ei.industry_id = i.id
                group by e.id
                having count(distinct i.id) = 4
                order by e.name
                offset 2
                limit 1
            '''

In [31]:
df_5_5 = pd.read_sql_query(query_5_5, connection)
df_5_5

C:\Users\levte\AppData\Local\Temp\ipykernel_27752\480638788.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_5_5 = pd.read_sql_query(query_5_5, connection)


,name
0,2ГИС


6. Количество работодателей, у которых в качестве сферы деятельности указана "Разработка программного обеспечения"


In [32]:
# cоединяем таблицу employers с таблицей employers_industries через left join по id работодателя
# cоединяем полученную таблицу с таблицей industries через left join по id сферы деятельности
# считаем по количеству id работодателей, у которых в сфере деятельности указана "Разработка программного обеспечения"
query_5_6 = f'''select count(e.id)
                from employers as e 
                left join employers_industries as ei on e.id = ei.employer_id
                left join industries as i on ei.industry_id = i.id
                where i.name = 'Разработка программного обеспечения'
            '''

In [33]:
df_5_6 = pd.read_sql_query(query_5_6, connection)
df_5_6

C:\Users\levte\AppData\Local\Temp\ipykernel_27752\3865320243.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_5_6 = pd.read_sql_query(query_5_6, connection)


,count
0,3553


7. Cписок регионов-миллионников, в которых представлены вакансии компании "Яндекс", вместе с количеством вакансий в этих регионах

In [34]:
# импортируем urllib.parse
# из таблицы с информацией о городах-миллионниках, размещенной на сайте wikipedia, извлекаем только названия городов
# сохраняем название городов-миллионников в форме списка cities
# корректируем название города Воронеж

import urllib.parse as req

path_common = req.urlparse('https://ru.wikipedia.org/wiki/Города-миллионеры_России#Список_городов-миллионеров')
url = path_common.scheme + '://' + req.quote(path_common.netloc) +  req.quote(path_common.path) + '#'+ req.quote(path_common.fragment)

cities = list(pd.read_html(url)[1]['Город'])
cities[13] = 'Воронеж'
cities

['Москва',
 'Санкт-Петербург',
 'Новосибирск',
 'Екатеринбург',
 'Казань',
 'Нижний Новгород',
 'Челябинск',
 'Красноярск',
 'Самара',
 'Уфа',
 'Ростов-на-Дону',
 'Омск',
 'Краснодар',
 'Воронеж',
 'Пермь',
 'Волгоград']

In [35]:
# Таблица 1:
# cоединяем таблицу vacancies с таблицей employers через left join по id работодателя
# cоединяем полученную таблицу с таблицей areas через left join по id регионов
# создаем условие, при котором в рассчет берутся только работодатель "Яндекс" и регион в cities
# считаем по количеству id вакансий
# группируем по имени регионов

# Таблица 2:
# для вывода дополнительной строки с общим количеством вакансий производим вышеперечисленные операции кроме группировки 
# считаем по количеству всех id вакансий

# объединяем две таблицы
# сортируем объдиненную таблицу по количеству вакансий в порядке возрастания

query_5_7 = f'''select
                    a.name,
                    count(v.id) as cnt
                from vacancies as v
                left join employers as e on e.id = v.employer_id
                left join areas as a on v.area_id = a.id
                where e.name = 'Яндекс' and a.name in {tuple(cities)}
                group by a.name
                
                union all

                select 
                    'total', 
                    count(*)
                from vacancies as v
                left join employers as e on e.id = v.employer_id
                left join areas as a on v.area_id = a.id
                where e.name = 'Яндекс' and a.name in {tuple(cities)}
                
                order by cnt
            '''

In [36]:
df_5_7 = pd.read_sql_query(query_5_7, connection)
df_5_7

C:\Users\levte\AppData\Local\Temp\ipykernel_27752\43835236.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_5_7 = pd.read_sql_query(query_5_7, connection)


,name,cnt
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


***

Выводы по анализу работодателей:
1. "Яндекс" с огромным отрывом лидирует по количеству имеющихся вакансий, в 5-6 раз опережая по данному показателю ближайших "преследователей".
2. Также, география вакансий компании "Яндекс" является самой обширной, покрывая почти четверть представленных в базе данных регионов.
3. При этом, четверть от общего числа вакансий компании "Яндекс" размещена в городах-миллионниках РФ.
4. Вакансии 36% работодателей невозможно отыскать по сфере деятельности, что делает портал менее пригодным для поиска работы.
5. Исходя из имеющейся информации, минимум 15% работодателей из общего числа являются IT компаниями.

# Юнит 6. Предметный анализ

1. Количество вакансий, имеющих отношение к данным, т.е. в их названии содержатся слова 'data' или 'данн'


In [37]:
# создаем условие, при котором из таблицы vacancies в рассчет берутся только те названия вакансий, в которых содержатся необходимые нам слова
# считаем по количеству id вакансий
query_6_1 = f'''select count(v.id)
                from vacancies as v
                where lower(v.name) like '%data%'
                or lower(v.name) like '%данн%'
            '''

In [38]:
df_6_1 = pd.read_sql_query(query_6_1, connection)
df_6_1

C:\Users\levte\AppData\Local\Temp\ipykernel_27752\1300834887.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_6_1 = pd.read_sql_query(query_6_1, connection)


,count
0,1771


2. Количество вакансий для начинающего дата-сайентиста. 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.

In [54]:
# создаем условие, при котором из таблицы vacancies в рассчет берутся только те названия вакансий, 
# в которых содержатся необходимые нам слова,
# а также вакансии без требуемого опыта работы или с типом трудоустройства "Стажировка"
# считаем по количеству id вакансий
query_6_2 = f'''select count(v.id)
                from vacancies as v
                where 
                    ((v.name like '%ML%' and v.name not like '%HTML%')
                    or v.name ilike '%machine learning%'
                    or v.name ilike '%машинн%обучен%'
                    or v.name ilike '%data scientist%'
                    or v.name ilike '%data science%'
                    or v.name ilike '%исследователь данных%')
                    and 
                    (v.name ilike '%junior%'
                    or v.experience = 'Нет опыта'
                    or v.employment = 'Стажировка')
            '''

In [55]:
df_6_2 = pd.read_sql_query(query_6_2, connection)
df_6_2

C:\Users\levte\AppData\Local\Temp\ipykernel_27752\2048769617.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_6_2 = pd.read_sql_query(query_6_2, connection)


,count
0,51


3. Количество вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres

In [56]:
# создаем условие, при котором из таблицы vacancies в рассчет берутся только те названия вакансий, 
# в которых содержатся необходимые нам слова,
# а также те ключевые навыки, в которых встречаются SQL и postgres
# считаем по количеству id вакансий
query_6_3 = f'''select count(v.id)
                from vacancies as v
                where 
                    ((v.name like '%ML%' and v.name not like '%HTML%')
                    or v.name ilike '%machine learning%'
                    or v.name ilike '%машинн%обучен%'
                    or v.name ilike '%data scientist%'
                    or v.name ilike '%data science%'
                    or v.name ilike '%исследователь данных%')
                    and
                    (v.key_skills ilike '%SQL%'
                    or v.key_skills ilike '%postgres%')
            '''

In [57]:
df_6_3 = pd.read_sql_query(query_6_3, connection)
df_6_3

C:\Users\levte\AppData\Local\Temp\ipykernel_27752\3192746363.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_6_3 = pd.read_sql_query(query_6_3, connection)


,count
0,201


4. Количество вакансий, в которых в качестве ключевого навыка указан Python

In [58]:
# создаем условие, при котором из таблицы vacancies в рассчет берутся только те названия вакансий, 
# в которых содержатся необходимые нам слова.
# а также те ключевые навыки, в которых встречается Python
# считаем по количеству id вакансий
query_6_4 = f'''select count(v.id)
                from vacancies as v
                where 
                    ((v.name like '%ML%' and v.name not like '%HTML%')
                    or v.name ilike '%machine learning%'
                    or v.name ilike '%машинн%обучен%'
                    or v.name ilike '%data scientist%'
                    or v.name ilike '%data science%'
                    or v.name ilike '%исследователь данных%')
                    and
                    v.key_skills ilike '%Python%'
            '''

In [59]:
df_6_4 = pd.read_sql_query(query_6_4, connection)
df_6_4

C:\Users\levte\AppData\Local\Temp\ipykernel_27752\1319374162.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_6_4 = pd.read_sql_query(query_6_4, connection)


,count
0,351


5. Среднее количество требуемых ключевых навыков в вакансиях для DS

In [66]:
# создаем условие, при котором из таблицы vacancies в рассчет берутся только те названия вакансий, 
# в которых содержатся необходимые нам слова.
# среднее количество ключевых навыков высчитывам через количество табуляций + 1 (т.к. перед первым по списку навыком табуляции нет)
query_6_5 = f'''select round(avg(length(key_skills) - length(replace(key_skills, CHR(9), '')) + 1), 2) as ks_avg_cnt
                from vacancies as v
                where 
                    (v.name like '%ML%' and v.name not ilike '%HTML%')
                    or v.name ilike '%machine learning%'
                    or v.name ilike '%машинн%обучен%'
                    or v.name ilike '%data scientist%'
                    or v.name ilike '%data science%'
                    or v.name ilike '%исследователь данных%'
            '''

In [67]:
df_6_5 = pd.read_sql_query(query_6_5, connection)
df_6_5

C:\Users\levte\AppData\Local\Temp\ipykernel_27752\1566661722.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_6_5 = pd.read_sql_query(query_6_5, connection)


,ks_avg_cnt
0,6.41


6. Средняя зарплата для DS для каждого типа требуемого опыта

In [73]:
# создаем условие, при котором из таблицы vacancies в рассчет берутся только те названия вакансий, 
# в которых содержатся необходимые нам слова.
# среднюю зарплату высчитываем при помощи метода coalesce(), чтобы исключить учет null-евых значений.
# группируем по опыту работы
query_6_6 = f'''select
                    v.experience,
                    round(avg(coalesce((v.salary_from + v.salary_to)/2, v.salary_from, v.salary_to))) as avg_salary
                from vacancies as v
                where 
                    (v.name like '%ML%' and v.name not ilike '%HTML%')
                    or v.name ilike '%machine learning%'
                    or v.name ilike '%машинн%обучен%'
                    or v.name ilike '%data scientist%'
                    or v.name ilike '%data science%'
                    or v.name ilike '%исследователь данных%'
                group by v.experience
            '''

In [74]:
df_6_6 = pd.read_sql_query(query_6_6, connection)
df_6_6

C:\Users\levte\AppData\Local\Temp\ipykernel_27752\158571054.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_6_6 = pd.read_sql_query(query_6_6, connection)


,experience,avg_salary
0,Более 6 лет,NaN
1,Нет опыта,74643.0
2,От 1 года до 3 лет,139675.0
3,От 3 до 6 лет,243115.0


***

Выводы по предметному анализу:
1. Количество вакансий, имеющих отношение к данным, составляет всего 3.5% от общего числа. Этот показатель не отражает реальную ситуацию на рынке труда, учитывая, что на IT компании приходится 15% всех вакансий, как мы выяснили ранее. Для более реалистичной картины, следует добавить в поиск дополнительные ключевые слова (например, "информация"). 
2. При этом, всего на Data Science приходится 480 вакансий или 27% от общего числа вакансий, связанных с данными.
3. В разрезе DS вакансий: 10% вакансий для начинающих специалистов, 42% вакансий для специалистов с навыком работы с базами данных, 73% вакансий для знающих язык Python.
4. Согласно требованиям рынка, в среднем дата-сайентисту нужно обладать как минимум 6-7 ключевыми навыками.
5. Средняя предлагаемая зарплата для дата-сайентиста без опыта работы примерно равна средней минимальной зарплате по всем сферам деятельности вне зависимости от стажа и на 85% выше средней зарплаты для всех неопытных соискателей в целом по рынку труда.

# Общий вывод по проекту

1. На рынке преобладают вакансии для специалистов, в той или иной степени имеющих отношение к компьютерным сетям, серверам, базам данных, т.е. сисадмины, системные аналитики и т.д. Отсюда можно сделать вывод, что на рынке имеется острая нехватка данной категории специалистов.
2. В 10% вакансий требуется знание английского языка, а знание компьютера требуется лишь в 1% вакансий от общего числа. Можно сделать вывод, что английский язык все еще значительно преобладает над знанием ПК; либо работодатели уже не указывают данное требование, подразумевая, что все люди по умолчанию умеют работать с ПК.
3. Примечательно, что несмотря на то, что уровень зарплат для DS профессий значительно выше, чем для всех остальных профессий, показатели прироста зарплаты дата-сайентиста в зависимости от опыта работы примерно совпадают с общей тенденцией на рынке труда по всем сферам деятельности.
4. Интересно было бы еще исследовать динамику роста вакансий для удаленной работы по сравнению с полным рабочим днем за последние пять лет, т.е. до пандемии и после. Но, к сожалению, это не представляется возможным ввиду нехватки данных.

In [75]:
# закрываем соединение
connection.close()